In [6]:
import numbers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

%config Completer.use_jedi = False


def get_mape(y_predict, y_true):
    return (abs(y_predict - y_true) / y_true).mean()


def process_rooms_number(x):
        
    if pd.isna(x):
        return 1
    
    if isinstance(x, int):
        return x
    
    if x.isdigit():
        return int(x)
    
    if x == 'Студия':
        return 1
    
    if x == 'Своб. планировка':
        return 1
    
    if x == '> 9':
        return 10

    return 1

<h3>Реализуем линейную регрессию</h3>

<p>Чаще всего алгоритмы машинного обучения реализуются в виде классов с обязательными методами <code>.fit()</code>, <code>.predict()</code>. </p>

<p><code>.fit()</code> – обучить алгоритм на обучающей выборке;</p>

<p><code>.predict()</code> – сделать предсказание на тестовых данных.</p>

<p> </p>

In [28]:
class LinearRegression:
    
    def __init__(self, max_iter=1e4, lr=0.001, tol=0.001, print_every=100):
        
        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.print_every = print_every
        
        self.weights = None
        self.bias = None
        
    def fit(self, X_train, y_train, X_val, y_val):
        
        self.check_regression_X_y(X_train, y_train)
        self.check_regression_X_y(X_val, y_val)
        
        n, m = X_train.shape
        
        self.weights = np.zeros((m, 1))
        self.bias = np.median(y_train)
        
        n_iter = 0
        gradient_norm = np.inf
        
        while n_iter < self.max_iter and gradient_norm > self.tol:
            

            dJdw, dJdb = self.grads(X_train, y_train)
                
            gradient_norm = np.linalg.norm(np.hstack([dJdw.flatten(), [dJdb]]))
                
            self.weights = self.weights - self.lr * dJdw
            self.bias = self.bias - self.lr * dJdb
            
            n_iter += 1
            
            if n_iter % self.print_every == 0:
                self.print_metrics(X_train, y_train, X_val, y_val, n_iter, gradient_norm)
        
        return self

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias
    
    def grads(self, X, y):
        
        y_hat = self.predict(X)
        
        dJdw = 2 * (X.T @ (y_hat - y)) / len(X)
        dJdb = 2 * (y_hat - y).mean()
        
        self.check_grads(dJdw, dJdb)
        
        return dJdw, dJdb
    
    def print_metrics(self, X_train, y_train, X_val, y_val, n_iter, gradient_norm):
        
        train_preds = self.predict(X_train)
        val_preds = self.predict(X_val)
        
        MAPE_train = get_mape(train_preds, y_train)
        MAPE_val = get_mape(val_preds, y_val)
        MSE_train = ((y_train - self.predict(X_train)) ** 2).mean()
        
        print(f'{n_iter} completed. MAPE on train: {round(MAPE_train, 3)}, val: {round(MAPE_val, 3)},  grad norm: {gradient_norm}, MSE: {round(MSE_train, 2)}')
        
        
    def check_grads(self, dJdw, dJdb):
        
        if not isinstance(dJdb, numbers.Real):
            raise ValueError(f'Производная по параметру b должна быть действительным '
                             f'числом, как и сам параметр b, а у нас {dJdb} типа {type(dJdb)}')
            
        if dJdw.shape != self.weights.shape:
            raise ValueError(f'Размерность градиента по параметрам w должна совпадать с самим вектором w, '
                             f'а у нас dJdw.shape = {dJdw.shape} не совпадает с weight.shape = {self.weights.shape}')
            
        
    @staticmethod
    def check_regression_X_y(X, y):
        
        if X.shape[0] == 0:
            raise ValueError(f'X и y не должны быть пустыми, а у нас X.shape = {X.shape} и y.shape = {y.shape}')
            
        if np.isnan(X).any():
            raise ValueError(f'X не должен содержать "not a number" (np.nan)')
            
        if np.isnan(y).any():
            raise ValueError(f'y не должен содержать "not a number" (np.nan)')
        
        if X.shape[0] != y.shape[0]:
            raise ValueError(f'Длина X и y должна быть одинаковой, а у нас X.shape = {X.shape}, y.shape = {y.shape}')
            
        if y.shape[1] != 1:
            raise ValueError(f'y - вектор ответов должен быть размерности (m, 1), а у нас y.shape = {y.shape}')
                    
        if np.any([(not isinstance(value, numbers.Real)) for value in y.flatten()]):
            raise ValueError(f'Ответы на объектах должны быть действительными числами!')
       

<h3>Тестируем модель на простой задаче</h3>

In [29]:
X = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 1],
])
y = np.array([[1], [2], [3], [4]])
model = LinearRegression(lr=0.1)
model.fit(X, y, X, y)
model.predict(X)

100 completed. MAPE on train: 0.027, val: 0.027,  grad norm: 0.04184749716482497, MSE: 0.0
200 completed. MAPE on train: 0.002, val: 0.002,  grad norm: 0.002752354006169089, MSE: 0.0


array([[1.0010693 ],
       [2.00106431],
       [3.00105932],
       [3.99815656]])

<h3>Решаем задачу предсказания цены</h3>

In [12]:
data = pd.read_csv('real_estate_novosibirsk.csv')

In [13]:
data

,area,area_raw,item_id,type_of_house,floor,floors_in_house,rooms_number,price,district,price_order_id
0,91.0,91.0,198518500076,Кирпичный,3.0,16.0,3,5100000.0,Кировский,1
1,91.0,91.0,198518500076,Кирпичный,3.0,16.0,3,5100000.0,Кировский,2
2,91.0,91.0,198518500076,Кирпичный,3.0,16.0,3,5100000.0,Кировский,3
3,18.7,18.7,257626750244,Кирпичный,4.0,5.0,Студия,1300000.0,Ленинский,1
4,NaN,NaN,367907500565,Кирпичный,4.0,4.0,4,3500000.0,Кировский,1
...,...,...,...,...,...,...,...,...,...,...
124571,33.0,33.0,832255750867,Кирпичный,10.0,10.0,Студия,2950000.0,Заельцовский,2
124572,33.0,33.0,832255750867,Кирпичный,10.0,10.0,Студия,2950000.0,Заельцовский,3
124573,51.0,51.0,833043000375,Панельный,4.0,10.0,2,3300000.0,Октябрьский,1
124574,51.0,51.0,833043000375,Панельный,4.0,10.0,2,3300000.0,Октябрьский,2


In [16]:
data.describe()

,area,area_raw,item_id,floor,floors_in_house,price,price_order_id
count,115768.000000,115768.000000,1.245760e+05,124294.000000,124288.000000,1.245760e+05,124576.000000
mean,51.323927,51.323927,7.085322e+11,21.003387,26.073475,3.501932e+06,1.986073
std,26.894466,26.894466,9.170946e+10,275.906792,279.251892,4.300868e+06,0.817022
min,10.000000,10.000000,4.023000e+03,1.000000,1.000000,0.000000e+00,1.000000
25%,34.600000,34.600000,6.750189e+11,3.000000,5.000000,2.170000e+06,1.000000
50%,44.500000,44.500000,7.010004e+11,5.000000,10.000000,2.900000e+06,2.000000
75%,61.000000,61.000000,7.730338e+11,9.000000,16.000000,4.000000e+06,3.000000
max,824.000000,824.000000,8.336540e+11,5144.000000,5207.000000,1.250000e+09,3.000000


<p>Чистим данные:</p>

In [17]:
data = data.drop_duplicates(subset=['item_id'], keep='last')
data = data.dropna(subset=['area'])
data['rooms_number'] = data['rooms_number'].apply(process_rooms_number).copy()
data = data[(data.price > 970000) & (data.price < 12700000)]
data = data[(data.floor < 59)]
data = data[(data.floors_in_house < 59)]

data = data.dropna(axis=0)

In [18]:
train, val, train_price, val_price = train_test_split(data.drop('price', axis=1), data['price'], random_state=42)

In [19]:
test_data = pd.read_csv('test_price_estimator.csv')
test, test_price = test_data.drop('price', axis=1), test_data['price']

y_train = train_price.values.reshape(-1, 1)
y_val = val_price.values.reshape(-1, 1)

### Делаем бейзлайн

In [22]:
get_mape(y_predict=np.median(y_train), y_true=y_val)

0.37614684131639187

### Обучаем модель

1) Начинаем с простого

In [32]:
X_train = train[['area']].values
X_val = val[['area']].values


model = LinearRegression(lr=1e-4, max_iter=140000, print_every=10000, tol=0.1)
model.fit(X_train, y_train, X_val, y_val)

10000 completed. MAPE on train: 0.402, val: 0.392,  grad norm: 622743.269729813, MSE: 1960592336205.56
20000 completed. MAPE on train: 0.346, val: 0.337,  grad norm: 432166.80283102504, MSE: 1685452015560.34
30000 completed. MAPE on train: 0.311, val: 0.303,  grad norm: 299911.94533538935, MSE: 1552944973226.96
40000 completed. MAPE on train: 0.289, val: 0.281,  grad norm: 208130.69020025225, MSE: 1489129839818.8
50000 completed. MAPE on train: 0.275, val: 0.267,  grad norm: 144437.00851858748, MSE: 1458396595908.22
60000 completed. MAPE on train: 0.266, val: 0.258,  grad norm: 100235.33487409202, MSE: 1443595525522.21
70000 completed. MAPE on train: 0.26, val: 0.253,  grad norm: 69560.58187835386, MSE: 1436467358863.03
80000 completed. MAPE on train: 0.257, val: 0.249,  grad norm: 48273.14197466467, MSE: 1433034447688.64
90000 completed. MAPE on train: 0.254, val: 0.246,  grad norm: 33500.24069926941, MSE: 1431381164355.82
100000 completed. MAPE on train: 0.252, val: 0.245,  grad norm

<p>Для того, чтобы начать ориентироваться в метрике решения задачи, очень важно построить одну или несколько простых моделей. Часто есть соблазн добавить все признаки сразу и обучить модель — мы так поступать не будем. Наоборот, мы будем постепенно добавлять признаки и следить за тем, что модель решает задачу лучше и лучше. </p>

2) Увеличиваем количество признаков

In [35]:
X_train = train[['area', 'floors_in_house', 'floor']].values
X_val = val[['area', 'floors_in_house', 'floor']].values

model = LinearRegression(lr=1e-4, max_iter=120000, print_every=10000, tol=0.1)
model.fit(X_train, y_train, X_val, y_val)

10000 completed. MAPE on train: 0.397, val: 0.388,  grad norm: 559032.9682965259, MSE: 1997469366974.96
20000 completed. MAPE on train: 0.355, val: 0.346,  grad norm: 445631.8374336207, MSE: 1746215385450.28
30000 completed. MAPE on train: 0.324, val: 0.314,  grad norm: 355234.3883037859, MSE: 1586557441290.42
40000 completed. MAPE on train: 0.3, val: 0.291,  grad norm: 283174.27085169335, MSE: 1485103689279.53
50000 completed. MAPE on train: 0.284, val: 0.275,  grad norm: 225731.71492567804, MSE: 1420635467223.62
60000 completed. MAPE on train: 0.271, val: 0.262,  grad norm: 179941.51435450805, MSE: 1379669494327.87
70000 completed. MAPE on train: 0.262, val: 0.254,  grad norm: 143439.96189837353, MSE: 1353637894854.89
80000 completed. MAPE on train: 0.256, val: 0.247,  grad norm: 114342.83379916062, MSE: 1337096259853.41
90000 completed. MAPE on train: 0.251, val: 0.242,  grad norm: 91148.12544697641, MSE: 1326584969957.21
100000 completed. MAPE on train: 0.248, val: 0.239,  grad nor

Делаем новые признаки
##### One hot encoding

In [36]:
ohe_example = pd.DataFrame({'feature': ['a', 'b', 'a', 'c']})
ohe_example

,feature
0,a
1,b
2,a
3,c


In [37]:
ohe = OneHotEncoder(sparse=False)
ohe.fit_transform(ohe_example)

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [41]:
train.district.value_counts()

Октябрьский        5335
Кировский          4801
Ленинский          4381
Калининский        2713
Заельцовский       2710
Дзержинский        2104
Центральный        1702
Первомайский       1590
Советский          1346
Железнодорожный    1039
Name: district, dtype: int64

In [42]:
ohe_house_type_transformer = OneHotEncoder(sparse=False)
train_ohe_house_type = ohe_house_type_transformer.fit_transform(train[['type_of_house']])
val_ohe_house_type = ohe_house_type_transformer.transform(val[['type_of_house']])

ohe_district_transformer = OneHotEncoder(sparse=False)
train_ohe_district = ohe_district_transformer.fit_transform(train[['district']])
val_ohe_district = ohe_district_transformer.transform(val[['district']])

X_train_extended = np.hstack([X_train, train_ohe_house_type, train_ohe_district])
X_val_extended = np.hstack([X_val, val_ohe_house_type, val_ohe_district])

model = LinearRegression(lr=1e-4, max_iter=120000, print_every=10000, tol=0.1)
model.fit(X_train_extended, y_train, X_val_extended, y_val)

10000 completed. MAPE on train: 0.334, val: 0.323,  grad norm: 665585.7962037667, MSE: 1576569971087.72
20000 completed. MAPE on train: 0.276, val: 0.267,  grad norm: 418913.4083964072, MSE: 1292546892672.62
30000 completed. MAPE on train: 0.249, val: 0.24,  grad norm: 288577.4454914911, MSE: 1170436821115.6
40000 completed. MAPE on train: 0.235, val: 0.227,  grad norm: 212763.52438280673, MSE: 1108687099083.68
50000 completed. MAPE on train: 0.226, val: 0.218,  grad norm: 164956.79739205152, MSE: 1073462035237.53
60000 completed. MAPE on train: 0.219, val: 0.212,  grad norm: 132575.83574238117, MSE: 1051527867703.31
70000 completed. MAPE on train: 0.215, val: 0.207,  grad norm: 109213.40688712966, MSE: 1037007016783.43
80000 completed. MAPE on train: 0.211, val: 0.204,  grad norm: 91477.18053209613, MSE: 1026987110131.22
90000 completed. MAPE on train: 0.209, val: 0.201,  grad norm: 77507.94039473408, MSE: 1019876084922.23
100000 completed. MAPE on train: 0.207, val: 0.199,  grad norm

### Задание на семинаре: попробовать улучшить метрику MAPE до 15.8% (топ-1 без ML с первой недели).

Варианты путей для улучшения:

    1) Делать новые признаки из существующих;
    2) Препроцессинг данных, целевой переменной - постпроцессинг ответов модели;
    3) Анализ ошибок модели –> генерация идей;
    4) Добавить регуляризацию;


<h3>Задание на семинаре: реализуем логистическую регрессию</h3>

<p>Мы получаем оптимальные веса алгоритма градиентным спуском:</p>

<p style="text-align:center"><br />
<br />
<span class="math-tex">\(\begin{bmatrix} w_{1}^{t+1}\\  ...\\ w_{m}^{t+1}\\  \end{bmatrix} = \begin{bmatrix} w_{1}^{t}\\  ...\\ w_{m}^{t}\\  \end{bmatrix} - \alpha \cdot  \begin{bmatrix} \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})x_{1}^{(i)}\\  ...\\ \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})x_{m}^{(i)}\\  \end{bmatrix}\)</span></p>

<p style="text-align:center"><span class="math-tex">\(b^{t+1} = b^{t} - \alpha \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})\)</span></p>

<p style="text-align:center">&nbsp;</p>

<p>&nbsp;</p>

In [ ]:
from sklearn.metrics import accuracy_score


class LogisticRegression:
    
    def __init__(self, max_iter=1e4, lr=0.001, tol=0.001, print_every=100):
        
        '''
        max_iter – максимальное количеств
        '''
        
        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.print_every = print_every
        
        self.weights = None
        self.bias = None
        
    def fit(self, X_train, y_train, X_val, y_val):
        
        '''
        Обучение модели.
        
        X_train – матрица объектов для обучения
        y_train – ответы на объектах для обучения
        
        X_val – матрица объектов для валидации
        y_val – ответы на объектах для валидации
        '''
        
        self.check_binary_clf_X_y(X_train, y_train)
        self.check_binary_clf_X_y(X_val, y_val)
                
        n, m = X_train.shape
        
        self.weights = 
        self.bias = 
        
        n_iter = 0
        gradient_norm = np.inf
        
        while n_iter < self.max_iter and gradient_norm > self.tol:
            
            dJdw, dJdb = self.grads(X_train, y_train)
            gradient_norm = np.linalg.norm(np.hstack([dJdw.flatten(), [dJdb]]))
                
            self.weights = 
            self.bias = 
            
            n_iter += 1
            
            if n_iter % self.print_every == 0:
                self.print_metrics(X_train, y_train, X_val, y_val, n_iter, gradient_norm)
        
        return self
    
    def predict(self, X):  
        
        '''
        Метод возвращает предсказанную метку класса на объектах X
        '''
        
        pass
        
    
    def predict_proba(self, X):
        
        '''
        Метод возвращает вероятность класса 1 на объектах X
        '''
        pass
    
    def grads(self, x, y):
        
        '''
        Рассчёт градиентов
        '''
        y_hat = 
        
        dJdw = 
        dJdb = 
        
        self.check_grads(dJdw, dJdb)
        
        return dJdw, dJdb
    
    @staticmethod
    def sigmoid(x):
        '''
        Сигмоида от x
        '''
        pass
    
    def print_metrics(self, X_train, y_train, X_val, y_val, n_iter, gradient_norm):
        
        train_preds = self.predict(X_train)
        val_preds = self.predict(X_val)
        
        train_acc = accuracy_score(train_preds, y_train)
        val_acc = accuracy_score(val_preds, y_val)
        
        print(f'{n_iter} completed. accuracy_score on train: {train_acc}, val: {val_acc}, grad_norm: {gradient_norm}')
        
    def check_grads(self, dJdw, dJdb):
        
        if not isinstance(dJdb, numbers.Real):
            raise ValueError(f'Производная по параметру b должна быть действительным'
                             f' числом, как и сам параметр b, а у нас {dJdb} типа {type(dJdb)}')
            
        if dJdw.shape != self.weights.shape:
            raise ValueError(f'Размерность градиента по параметрам w должна совпадать с самим вектором w, '
                             f'а у нас dJdw.shape = {dJdw.shape} не совпадает с weight.shape = {self.weights.shape}')
    
    @staticmethod
    def check_binary_clf_X_y(X, y):
        
        if X.shape[0] == 0:
            raise ValueError(f'X и y не должны быть пустыми, а у нас X.shape = {X.shape} и y.shape = {y.shape}')
            
        if np.isnan(X).any():
            raise ValueError(f'X не должен содержать "not a number" (np.nan)')
            
        if np.isnan(y).any():
            raise ValueError(f'y не должен содержать "not a number" (np.nan)')
        
        if X.shape[0] != y.shape[0]:
            raise ValueError(f'Длина X и y должна быть одинаковой, а у нас X.shape = {X.shape}, y.shape = {y.shape}')
            
        if y.shape[1] != 1:
            raise ValueError(f'y - вектор ответов должен быть размерности (m, 1), а у нас y.shape = {y.shape}')

                    
        if sorted(np.unique([1, 0, 0])) != [0, 1]:
            raise ValueError(f'Ответы на объектах должны быть только 0 или 1, а у нас np.unique(y) = {np.unique(y)}')


<h2>Домашнее задание</h2>

<p>Воспользуемся реализованной моделью логистической регрессии, чтобы решить задачу определения пола пользователя Авито.</p>

<p><a href="https://stepik.org/media/attachments/lesson/527992/binary_clf_data.csv" rel="noopener noreferrer nofollow">Данные</a> даны в сыром виде &ndash; айтемы и их категории, которые выкладывали покупатели на Авито. Целевая переменная: <em>gender.</em></p>

<p>Вам необходимо разбить данные на train, val. Перед загрузкой файла с ответом убедитесь, что точность (<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html" rel="noopener noreferrer nofollow">accuracy</a>)&nbsp;на валидации не менее 0.7.</p>

<p>&nbsp;</p>

<p><strong>План действий</strong></p>

<p>Сначала нужно преобразовать категории с помощью one-hot encoding. Далее необходимо агрегировать категории, в которых пользователи выкладывали объявления, чтобы получить вектор признаков для каждого объекта. В результате у каждого пользователя будет вектор признаков, содержащий количество айтемов, выложенных в каждой из возможных категорий.</p>

<ul>
	<li>Убедитесь, что для каждого пользователя в выборке есть только один объект, каждый признак означает количество айтемов, выложенное этим пользователем в категории;</li>
	<li>Убедитесь, что после one-hot энкодинга каждая категория соответствует признаку,&nbsp;<strong>одинаковому в train, val и test.</strong></li>
</ul>

<p>Попробуйте варианты отбора признаков. Для борьбы с переобучением на редких категориях используйте регуляризацию. В качестве&nbsp;ответа загрузите файл с предсказанием пола для пользователей:</p>

<p style="text-align:center">&nbsp;</p>

<table align="center" border="1" cellpadding="1" cellspacing="1" style="width:500px">
	<thead>
		<tr>
			<th style="text-align:center">user_id</th>
			<th style="text-align:center">gender</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td style="text-align:center">15424171</td>
			<td style="text-align:center">male</td>
		</tr>
		<tr>
			<td style="text-align:center">15454025</td>
			<td style="text-align:center">female</td>
		</tr>
	</tbody>
</table>

<p style="text-align:center">&nbsp;</p>

<p>Такой файл можно сформировать с помощью&nbsp;<code>test_predictions.to_csv(&#39;test_predictions.csv&#39;, index=False)</code>.</p>

<p>После того, как получилось обучить модель, ответьте на вопрос: какие из категорий вносят наибольший вклад в вероятность класса &quot;мужчина&quot; и класса &quot;женщина&quot;?</p>

<p>Например, если вы закодировали &quot;мужчина&quot; как 1, большие положительные веса при признаках будут означать большой вклад в вероятность класса 1, большие по модулю отрицательные веса будут вносить наибольший вклад в вероятность класса 0. Согласуется ли полученный результат с вашим жизненным опытом?</p>
